# RGI06 (Iceland)

F. Maussion & S. Galos

Strictly equivalent to RGI6

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
from utils import mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist
import os

## Files and storage paths

In [ ]:
# Region of interest
reg = 6

# go down from rgi7_scripts/workflow
data_dir = '../../rgi7_data/'

# Level 2 GLIMS files
l2_dir = os.path.join(data_dir, 'l2_sel_reg_tars')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_rgi7a'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_rgi7a_tar'))

# RGI v6 file for comparison later 
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', '06_rgi60_Iceland.zip')

In [ ]:
# Specific to this region: RGI is made up by a series of spatially overlapping datasets
# selection of subsets is based on boxes which define areas for respective subset selection

support_dir = os.path.join(data_dir, 'l0_support_data')
box_file = os.path.join(support_dir, 'Iceland.tar.gz')

### Load the input data

In [ ]:
# Read L2 files
shp = gpd.read_file('tar://' + l2_dir + f'/RGI{reg:02d}.tar.gz/RGI{reg:02d}/RGI{reg:02d}.shp')

### List of submissions 

In [ ]:
sdf, _ = submission_summary(shp)
sdf

In [ ]:
# # Write out selection in intermediate shape files for manual GIS review
# tmp_output_dir = mkdir(os.path.join(data_dir, 'l0_tmp_data', f'rgi{reg:02d}_inventories'))
# tmp_output_dir_tar = mkdir(os.path.join(data_dir, 'l0_tmp_data'))
# for subid in shp.subm_id.unique():
#     s_loc = shp.loc[shp.subm_id == subid]
#     s_loc.to_file(tmp_output_dir + f'/subm_{int(subid):03d}.shp')
# print('Taring...')
# print(subprocess.run(['tar', '-zcvf', f'{tmp_output_dir_tar}/rgi{reg:02d}_inventories.tar.gz', '-C', 
#                       os.path.join(data_dir, 'l0_tmp_data'), f'rgi{reg:02d}_inventories']))

## Apply selection criteria to create the RGI7 data subset

### Extract RGI6 from GLIMS data and do a check

In this special case RGI06 was made up from several files. Submission id 719 was added to GLIMS to document applied changes in glacier devides for RGI6 so data set serves as the baseline, we then add those parts of other submissions which are not covered by subm_id 719. To extract those we apply an overlay of representative points of each file and the polygons of subm_id 719.   

In [ ]:
# define subsets according to submission id
sub_719 = shp.loc[shp['subm_id']==719]
sub_437 = shp.loc[shp['subm_id']==437]
sub_438 = shp.loc[shp['subm_id']==438]
sub_439 = shp.loc[shp['subm_id']==439]
sub_452 = shp.loc[shp['subm_id']==452]

### Extract relevant subset of each submission based on overlay with manually defined boxes

In [ ]:
# load shapes
AOI_437 = gpd.read_file('tar://' + box_file + '/Iceland/AOI_437.shp')
AOI_438 = gpd.read_file('tar://' + box_file + '/Iceland/AOI_438.shp')
AOI_439 = gpd.read_file('tar://' + box_file + '/Iceland/AOI_439.shp')
AOI_452 = gpd.read_file('tar://' + box_file + '/Iceland/AOI_452.shp')

In [ ]:
## extract relevant subsets by overlay

# 437
sub437_ss = gpd.overlay(sub_437, AOI_437, how='intersection')

# 438
sub438_ss = gpd.overlay(sub_438, AOI_438, how='intersection')

# 439
sub439_ss = gpd.overlay(sub_439, AOI_439, how='intersection')

# 452
sub452_ss = gpd.overlay(sub_452, AOI_452, how='intersection')

In [ ]:
# append to subm 719 in order to create RGI regional subset
rgi7 = sub_719.append([sub437_ss, sub438_ss, sub439_ss, sub452_ss], sort=True)

### Some sanity checks 

In [ ]:
sdf, df_class = submission_summary(rgi7)
df_class

In [ ]:
# Nothing should change here
rgi7['is_rgi6'] = True

In [ ]:
# Check the orphaned rock outcrops
orphan_f = os.path.join(data_dir, 'l1_orphan_interiors', f'RGI{reg:02d}', f'RGI{reg:02d}.shp')
if os.path.exists(orphan_f):
    orphan_f = gpd.read_file(orphan_f)
    if np.any(np.isin(rgi7.subm_id.unique(), orphan_f.subm_id.unique())):
        print('Orphan rock outcrops detected.')

### Plots 

In [ ]:
plot_map(rgi7, reg)

In [ ]:
plot_map(rgi7, reg, is_rgi6=True)

In [ ]:
plot_date_hist(rgi7, reg)

### Text for github

In [ ]:
fgh = sdf.T
fgh

In [ ]:
print(fgh.to_markdown(headers=np.append(['subm_id'], fgh.columns)))

## Write out and tar 

In [ ]:
dd = mkdir(f'{output_dir}/RGI{reg:02d}/', reset=True)

print('Writing...')
rgi7.to_file(dd + f'RGI{reg:02d}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI{reg:02d}.tar.gz', '-C', output_dir, f'RGI{reg:02d}']))

### load reference data (here RGI6) to enable comparison

In [ ]:
# load reference data
from utils import open_zip_shapefile
ref_odf = open_zip_shapefile(rgi6_reg_file)

#### Number of elements (differences depict problems)

In [ ]:
print('Number of glaciers in new RGI subset:', len(rgi7))
print('Number of glaciers in reference data:', len(ref_odf))
print('Difference:', len(rgi7)-len(ref_odf))

In [ ]:
# how many nominals in RGI06 (identifiable via 'Status' attribute in RGI 06)
nom = ref_odf.loc[ref_odf.Status == 2]
len(nom)

#### Total area

In [ ]:
# add an area field to RGI_ss and reference data
ref_odf['area'] = ref_odf.to_crs({'proj':'cea'}).area

In [ ]:
# print and compare area values
Area_RGI = rgi7['area'].sum()/1000000
print('Area RGI [km²]:', Area_RGI)
Area_ref = ref_odf['area'].sum()/1000000
print('Area Ref [km²]:', Area_ref)
d = (Area_RGI - Area_ref)
print('Area difference:',d, 'km²')

### Find the missing glacier 

In [ ]:
# Rename things
df_ref = ref_odf

In [ ]:
from utils import haversine
import progressbar
import numpy as np

In [ ]:
def xy_coord(geom):
    """To compute CenLon CenLat ourselves"""
    x, y = geom.xy
    return x[0], y[0]

In [ ]:
# compute CenLon CenLat ourselves
rp = df_ref.representative_point()

coordinates = np.array(list(rp.apply(xy_coord)))
df_ref['CenLon'] = coordinates[:, 0]
df_ref['CenLat'] = coordinates[:, 1]

In [ ]:
df_ref_orig = df_ref.copy()

In [ ]:
# Loop over all RGI7 glaciers and find their equivalent in ref
df_ref = df_ref_orig.copy()
not_found = {}
to_drop = []
for i, (ref_area, lon, lat) in progressbar.progressbar(enumerate(zip(rgi7['area'].values, rgi7.CenLon.values, rgi7.CenLat.values)), max_value=len(rgi7)):
#     dist = haversine(lon, lat, df_ref.CenLon.values, df_ref.CenLat.values)
    dist = (lon - df_ref.CenLon.values)**2 + (lat - df_ref.CenLat.values)**2 
    found = False
    for j in np.argsort(dist)[:10]:
        s6 = df_ref.iloc[j]
        if np.allclose(s6['area'], ref_area, rtol=0.01):
            found = True
            to_drop.append(s6.name)
            break
    if not found:
        not_found[i] = df_ref.iloc[np.argsort(dist)[:10]]
    if len(to_drop) > 1000:
        df_ref.drop(labels=to_drop, inplace=True)
        to_drop = []
df_ref.drop(labels=to_drop, inplace=True)

In [ ]:
print(len(not_found), len(df_ref))

In [ ]:
pb_rgi7 = rgi7.iloc[list(not_found.keys())]

In [ ]:
rgi7.iloc[list(not_found.keys())].plot(edgecolor='k');

In [ ]:
df_ref.plot(edgecolor='k');

We have found the problem! Reported here: https://trello.com/c/Q7QOMnkA

In [ ]:
# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_problem_glaciers'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_problem_glaciers_tar'))

In [ ]:
dd = mkdir(f'{output_dir}/RGI{reg:02d}/', reset=True)

print('Writing...')
pb_rgi7.to_file(dd + f'RGI{reg:02d}_glims.shp')
df_ref.to_file(dd + f'RGI{reg:02d}_ref.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI{reg:02d}.tar.gz', '-C', output_dir, f'RGI{reg:02d}']))